<a href="https://colab.research.google.com/github/manojach87/miscellaneous/blob/master/DodgersSummary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     |████████████████████████████████| 212.3MB 60kB/s 
     |████████████████████████████████| 204kB 38.6MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=5ee67bd0e9d28e732e96c48a82ce21a5c9e0d679d658822d0550d5880090a24c
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


In [2]:
import pyspark as py

In [3]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/event-detection/Dodgers.data

--2021-04-01 20:59:44--  https://archive.ics.uci.edu/ml/machine-learning-databases/event-detection/Dodgers.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 961432 (939K) [application/x-httpd-php]
Saving to: ‘Dodgers.data’

Dodgers.data        100%[===================>] 938.90K  2.24MB/s    in 0.4s    

2021-04-01 20:59:45 (2.24 MB/s) - ‘Dodgers.data’ saved [961432/961432]



In [4]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/event-detection/Dodgers.events

--2021-04-01 20:59:46--  https://archive.ics.uci.edu/ml/machine-learning-databases/event-detection/Dodgers.events
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4140 (4.0K) [application/x-httpd-php]
Saving to: ‘Dodgers.events’

Dodgers.events      100%[===================>]   4.04K  --.-KB/s    in 0s      

2021-04-01 20:59:46 (89.7 MB/s) - ‘Dodgers.events’ saved [4140/4140]



Attribute Information:

1. Date: MM/DD/YY
2. Time: (H)H:MM (military time)
3. Count: Number of cars measured for the previous five minutes
Rows: Each five minute time slice is represented by one row

For .events file:
1. Date: MM/DD/YY
2. Begin event time: HH:MM:SS (military)
3. End event time: HH:MM:SS (military)
4. Game attendance
5. Away team
6. W/L score

In [5]:
sc=py.SparkContext()

In [6]:
trafficPath="Dodgers.data"
gamesPath="Dodgers.events"

In [7]:
traffic=sc.textFile(trafficPath)

In [8]:
traffic.take(10)

['4/10/2005 0:00,-1',
 '4/10/2005 0:05,-1',
 '4/10/2005 0:10,-1',
 '4/10/2005 0:15,-1',
 '4/10/2005 0:20,-1',
 '4/10/2005 0:25,-1',
 '4/10/2005 0:30,-1',
 '4/10/2005 0:35,-1',
 '4/10/2005 0:40,-1',
 '4/10/2005 0:45,-1']

In [9]:
games=sc.textFile(gamesPath)
games.take(10)

['04/12/05,13:10:00,16:23:00,55892,San Francisco,W 9-8�',
 '04/13/05,19:10:00,21:48:00,46514,San Francisco,W 4-1�',
 '04/15/05,19:40:00,21:48:00,51816,San Diego,W 4-0�',
 '04/16/05,19:10:00,21:52:00,54704,San Diego,W 8-3�',
 '04/17/05,13:10:00,15:31:00,53402,San Diego,W 6-0�',
 '04/25/05,19:10:00,21:33:00,36876,Arizona,L 4-2�',
 '04/26/05,19:10:00,22:00:00,44486,Arizona,L 3-2�',
 '04/27/05,19:10:00,22:17:00,54387,Arizona,L 6-3�',
 '04/29/05,19:40:00,22:01:00,40150,Colorado,W 6-3�',
 '04/30/05,19:10:00,21:45:00,54123,Colorado,W 6-2�']

In [10]:
 from datetime import datetime
 import csv
 from io import StringIO 

In [11]:
def parseTraffic(row):
  DATE_FMT= "%m/%d/%Y %H:%M"
  row=row.split(",")
  row[0]=datetime.strptime(row[0],DATE_FMT)
  row[1]=int(row[1])
  return(row)

In [12]:
trafficParsed = traffic.map(parseTraffic)
trafficParsed.take(10)

[[datetime.datetime(2005, 4, 10, 0, 0), -1],
 [datetime.datetime(2005, 4, 10, 0, 5), -1],
 [datetime.datetime(2005, 4, 10, 0, 10), -1],
 [datetime.datetime(2005, 4, 10, 0, 15), -1],
 [datetime.datetime(2005, 4, 10, 0, 20), -1],
 [datetime.datetime(2005, 4, 10, 0, 25), -1],
 [datetime.datetime(2005, 4, 10, 0, 30), -1],
 [datetime.datetime(2005, 4, 10, 0, 35), -1],
 [datetime.datetime(2005, 4, 10, 0, 40), -1],
 [datetime.datetime(2005, 4, 10, 0, 45), -1]]

In [13]:
## This performs group by and then sum of col 2 and then reverse sorty by col2
dailyTrend=trafficParsed.map(lambda x:(x[0].date(),x[1]))\
          .reduceByKey(lambda x,y: x+y ) \
          .sortBy(lambda x:-x[1])
dailyTrend.take(10)

## same as 
##select date(col1), sum(col2) from trafficParsed group by date(col1) order by col22 desc

[(datetime.date(2005, 7, 28), 7661),
 (datetime.date(2005, 7, 29), 7499),
 (datetime.date(2005, 8, 12), 7287),
 (datetime.date(2005, 7, 27), 7238),
 (datetime.date(2005, 9, 23), 7175),
 (datetime.date(2005, 7, 26), 7163),
 (datetime.date(2005, 5, 20), 7119),
 (datetime.date(2005, 8, 11), 7110),
 (datetime.date(2005, 9, 8), 7107),
 (datetime.date(2005, 9, 7), 7082)]

In [14]:
def parseGames(row):
  DATE_FMT= "%m/%d/%y"
  row=row.split(",")
  row[0]=datetime.strptime(row[0],DATE_FMT).date()
  return((row[0],row[4]))

In [15]:
gamesParsed=games.map(parseGames)
gamesParsed.take(10)

[(datetime.date(2005, 4, 12), 'San Francisco'),
 (datetime.date(2005, 4, 13), 'San Francisco'),
 (datetime.date(2005, 4, 15), 'San Diego'),
 (datetime.date(2005, 4, 16), 'San Diego'),
 (datetime.date(2005, 4, 17), 'San Diego'),
 (datetime.date(2005, 4, 25), 'Arizona'),
 (datetime.date(2005, 4, 26), 'Arizona'),
 (datetime.date(2005, 4, 27), 'Arizona'),
 (datetime.date(2005, 4, 29), 'Colorado'),
 (datetime.date(2005, 4, 30), 'Colorado')]

In [25]:
dailyTrendCombined=dailyTrend.leftOuterJoin(gamesParsed)
dailyTrendCombined.top(10)

[(datetime.date(2005, 10, 1), (-260, None)),
 (datetime.date(2005, 9, 30), (7079, None)),
 (datetime.date(2005, 9, 29), (6917, 'Arizona')),
 (datetime.date(2005, 9, 28), (6831, 'Arizona')),
 (datetime.date(2005, 9, 27), (6664, 'Arizona')),
 (datetime.date(2005, 9, 26), (6394, 'Pittsburgh')),
 (datetime.date(2005, 9, 25), (4991, 'Pittsburgh')),
 (datetime.date(2005, 9, 24), (5848, 'Pittsburgh')),
 (datetime.date(2005, 9, 23), (7175, 'Pittsburgh')),
 (datetime.date(2005, 9, 22), (6810, None))]

In [18]:
#dailyTrendCombined=dailyTrend.join(gamesParsed)
#dailyTrendCombined.top(10)

[(datetime.date(2005, 9, 29), (6917, 'Arizona')),
 (datetime.date(2005, 9, 28), (6831, 'Arizona')),
 (datetime.date(2005, 9, 27), (6664, 'Arizona')),
 (datetime.date(2005, 9, 26), (6394, 'Pittsburgh')),
 (datetime.date(2005, 9, 25), (4991, 'Pittsburgh')),
 (datetime.date(2005, 9, 24), (5848, 'Pittsburgh')),
 (datetime.date(2005, 9, 23), (7175, 'Pittsburgh')),
 (datetime.date(2005, 9, 14), (5631, 'Colorado')),
 (datetime.date(2005, 9, 13), (5662, 'Colorado')),
 (datetime.date(2005, 9, 12), (5726, 'Colorado'))]

In [26]:
def checkGameDay(row):
  if(row[1][0] == None or row[1][0] <= 0):
    return ( (row[0],row[1][0],"Regular Day",row[1][1]))
  else:
    return ( (row[0],row[1][0],"Game Day",row[1][1]))

In [27]:
dailyTrendCombined.map(checkGameDay).take(50)

[(datetime.date(2005, 9, 8), 7107, 'Game Day', None),
 (datetime.date(2005, 7, 25), 6898, 'Game Day', 'Cincinnati'),
 (datetime.date(2005, 9, 9), 6897, 'Game Day', 'San Diego'),
 (datetime.date(2005, 9, 22), 6810, 'Game Day', None),
 (datetime.date(2005, 9, 15), 6721, 'Game Day', None),
 (datetime.date(2005, 5, 13), 6706, 'Game Day', 'Atlanta'),
 (datetime.date(2005, 7, 15), 6698, 'Game Day', 'San Francisco'),
 (datetime.date(2005, 8, 2), 6649, 'Game Day', None),
 (datetime.date(2005, 7, 19), 6576, 'Game Day', None),
 (datetime.date(2005, 7, 8), 6563, 'Game Day', None),
 (datetime.date(2005, 6, 1), 6520, 'Game Day', 'Chicago Cubs'),
 (datetime.date(2005, 4, 15), 6459, 'Game Day', 'San Diego'),
 (datetime.date(2005, 5, 4), 6423, 'Game Day', 'Washington'),
 (datetime.date(2005, 9, 2), 6358, 'Game Day', None),
 (datetime.date(2005, 4, 25), 6280, 'Game Day', 'Arizona'),
 (datetime.date(2005, 6, 8), 6278, 'Game Day', 'Detroit'),
 (datetime.date(2005, 8, 1), 6233, 'Game Day', None),
 (dateti